In [29]:
import numpy as np
import pandas as pd

from pptx import Presentation

from funs.areas import *
from funs.car import *
from funs.conclusiones import conclusiones
from funs.correlacion import *
from funs.diapo import *
from funs.dimensiones import *
from funs.imp_rel import dict_ir
from funs.items import *
from funs.linea import *
from funs.otros import *
from funs.porcent import *
from funs.tri_count import tri_count

from data.texto.dimensiones import dict_dims
from data.texto.conclusiones import dict_c_pos, dict_c_neg 
from data.texto.items import dict_items

### __Versiones de los paquetes (15/12/2022)__

- Numpy = 1.23.5
- Pandas = 1.5.2
- pptx = 0.6.21

### __Creación DF__

In [30]:
df_par = pd.read_excel('data/2022/df_2022.xlsx', sheet_name='participantes_1', index_col='token')
df_res = pd.read_excel('data/2022/df_2022.xlsx', sheet_name='respuestas_lime', index_col='token')
df_imp = pd.read_excel('data/2022/df_2022.xlsx', sheet_name='importancia_relativa', index_col='token')
df_asi = pd.read_excel('data/2022/df_2022.xlsx', sheet_name='tasa_respuesta', index_col='areas')
df_mide = pd.read_excel('data/estudios_pasados.xlsx')
df_dims_mean = pd.read_excel('data/2022/df_2022.xlsx', sheet_name='promedio_dimensiones', index_col='token')

In [31]:
df_2022 = pd.merge(df_par, df_res, left_index=True, right_index=True)
df_2022 = tri_count(df_2022, dict_dims.keys(), dict_items.keys())

# df.to_excel('data/df.xlsx', sheet_name='data')
# df = pd.read_excel("data/df.xlsx", index_col=0)

In [32]:
df_2021 = pd.read_excel("data/2021/df.xlsx")

### __Definir orden y caracteristicas de los grupos__

In [33]:
segs = [s for s in df_2022.columns if s[:3] == 'seg']

In [34]:
dims = list(dict_dims.keys())
dims_drop = ['PBD', 'CFZ', 'IDS', 'ENL', 'IDC', 'SAT']
dims_glob = [d for d in dims if d not in dims_drop]
dims_org = ['AMF', 'APO', 'BNF', 'REM', 'COM', 'COO', 'TVP', 'STB', 'PRO', 'CFZ', 'PBD'] # las 2 últimas son 
dims_loc = ['AMI', 'ALS', 'CDT', 'CLA', 'DEI', 'EQI', 'LID', 'ORG', 'REC']               # exclusivas de Vitacura... en qué análisis se consideran?
dims_ind_res = ['IDS', 'ENL', 'IDC', 'SAT']

In [35]:
areas = get_areas(df_2022, segs)
areas_glob = ["Global Vitacura", "Área Salud", "Área Municipalidad", "Área Educación"]
areas_not  = get_areas_not(df_asi, areas, disclaimer=True)
areas_not.remove('Área Salud')

Cantidad: 65		Tasa de respuesta: 39%		Área Salud
Cantidad: 6		Tasa de respuesta: 60%		Dirección de Servicios de Salud, Educacion y demas incorporados a la gestion municipal
Cantidad: 42		Tasa de respuesta: 38%		Dirección de Desarrollo Comunitario
Cantidad: 5		Tasa de respuesta: 71%		Dirección de Asesoría Urbana
Cantidad: 6		Tasa de respuesta: 55%		Administración Municipal
Cantidad: 66		Tasa de respuesta: 34%		Colegio Antártica Chilena
Cantidad: 5		Tasa de respuesta: 31%		Subdirección de Administración
Cantidad: 6		Tasa de respuesta: 86%		Departamento de Atención al Vecino
Cantidad: 15		Tasa de respuesta: 37%		Departamento de Salud
Cantidad: 38		Tasa de respuesta: 33%		CESFAM
Cantidad: 4		Tasa de respuesta: 24%		Subdirección de Gestión y Recursos Internos
Cantidad: 6		Tasa de respuesta: 86%		Subdireccion de Aseo
Cantidad: 12		Tasa de respuesta: 29%		Asistentes Liceo
Cantidad: 5		Tasa de respuesta: 42%		Departamento de Adquisiciones y Abastecimiento
Cantidad: 6		Tasa de respuesta: 55%		D

Áreas que no cumplieron los criterios antes

In [36]:
areas_not_2021 = ['Alcaldía', 'Dirección de Asesoría Jurídica', 'Colegio Antártica Chilena', 'Administración Municipal', 'Dirección Innovación y Sustentabilidad', 'CESFAM - VITABOTICA', 'CESFAM - COSAM', 'Departamento Organizaciones Comunitarias', 'Asistentes de la Educación Colegio']

In [37]:
areas_nuevas = ['Anfitriones del Parque', 'Departamento de Administración de Servicios', 'Departamento de Adquisiciones y Abastecimiento', 'Departamento de Educación', 'Departamento de Patentes Comerciales', 'Departamento de Permisos de Circulación', 'Departamento de Salud', 'Subdirección de Fiscalización, Emergencia y Seguridad Interna', 'Subdirección de Gestion y Recursos Internos']

In [38]:
list_car1 = ['Masculino', 'Femenino']
list_car4 = ['Administrativos', 'Auxiliares', 'Directivos', 'Jefaturas', 'Profesionales', 'Técnicos']
list_car5 = ['Contrata', 'Estatuto', 'Honorarios', 'Indefinido Educación', 'Planta', 'Plazo Fijo', 'Titular']

##### __Se define el promedio de los resultados en apreciación positivas en cada una de las dimensiones en estudios anteriores realizados por MIDE__

In [39]:
df_ant_pub = df_mide[df_mide['tipo'] == 'publica'].iloc[:,4:].copy()
df_ant_prv = df_mide[df_mide['tipo'] == 'privada'].iloc[:,4:].copy()

dict_ant_pub = np.mean(df_ant_pub, axis=0).to_dict()
dict_ant_prv = np.mean(df_ant_prv, axis=0).to_dict()

list_ant_prv = [v for i, v in dict_ant_prv.items() if i in dims_glob]
list_ant_pub = [v for i, v in dict_ant_pub.items() if i in dims_glob]

__Crear los rangos de edad y antigüedad__

In [40]:
# Edad
age_bins = [-1, 30, 35, 40, 45, 50, 60, 85]
list_car2 = ["30 años o menos", "31-35", "36-40", "41-45", "46-50", "51-60", "61-85"]
df_2022["car_2"] = pd.cut(df_2022['car_2'], age_bins, labels=list_car2)

# Antigüedad
ant_bins = [-1, 0, 5, 10, 20, 45]
list_car3 = ["Menos de 1 año", "1-5", "6-10", "11-20", "21 años o más"]
df_2022["car_3"] = pd.cut(df_2022['car_3'], ant_bins, labels=list_car3)

__Crear los rangos de antigüedad__

In [41]:
df_2022.to_excel("data/test.xlsx")

### __Loop Maestro__

In [42]:
for i in segs:
    for j in df_2022[i].unique():
        if type(j) == str:
            print(j)

            if j not in areas_not:
                df_sub_2022 = df_2022[df_2022[i] == j]

                if (j in areas_not_2021) or (j in areas_nuevas):
                    unico = True

                else:
                    if j == 'DSP. Departamento de Operaciones y Fiscalización':
                        df_sub_2021 = df_2021[df_2021[i] == 'Departamento de Operaciones y Fiscalización']
                    elif j == 'Dirección de Servicios de Salud, Educacion y demas incorporados a la gestion municipal':
                        df_sub_2021 = df_2021[df_2021[i] == 'Dirección de Servicios de Salud, Educación y Demás']
                    elif j == 'Subdirección de Seguridad Pública':
                        df_sub_2021 = df_2021[df_2021[i] == 'Inspector-Motoristas']
                    elif j == 'Subdireccion de Ornato' or j == 'Subdireccion de Aseo':
                        df_sub_2021 = df_2021[df_2021[i] == 'Subdirección de Aseo y Ornato']
                    elif j == 'Asistentes Colegio':
                        df_sub_2021 = df_2021[df_2021[i] == 'Asistentes de la Educación Colegio']
                    elif j == 'Asistentes Liceo':
                        df_sub_2021 = df_2021[df_2021[i] == 'Asistentes de la Educación Liceo']
                    elif j == 'Departamento de Operaciones y Seguridad Ciudadana':
                        df_sub_2021 = df_2021[df_2021[i] == 'Subdirección de Operaciones']
                    elif j == 'COSAM':
                        df_sub_2021 = df_2021[df_2021[i] == 'CESFAM - COSAM']
                    elif j == 'DSP. Departamento de Operaciones y Fiscalización':
                        df_sub_2021 = df_2021[df_2021[i] == 'Departamento de Operaciones y Fiscalización']
                    else:
                        df_sub_2021 = df_2021[df_2021[i] == j]
                    unico = False

                # Segmentos y áreas previas y siguientes
                nxt_seg, sub_areas = nxt(df_sub_2022, i)
                ant_seg, sob_area  = ant(df_sub_2022, i)

                if j in ['Asistentes Colegio', 'Asistentes Liceo', 'Colegio Antártica Chilena', 'Liceo Amanda Labarca','Departamento de Educación']:
                    ant_seg, sob_area  = 'seg_2', 'Área Educación'
                elif j in ['CESFAM', 'COSAM', 'Departamento de Salud']:
                    ant_seg, sob_area  = 'seg_2', 'Área Salud'

                data_2022 = {
                    '<area>': j,
                    '<asi_all>': str(df_asi.loc[j][0]),
                    '<asi_yes>': str(df_asi.loc[j][1]),
                    '<asi_not>': str(df_asi.loc[j][0] - df_asi.loc[j][1]),

                    '<area_str10_pos>': str(round_int(get_per_dims(df_sub_2022, dims=dims_glob, signo='pos', promedio=True)*10)),
                    '<idc_str_pos>': get_per_dims(df_sub_2022, dims=['IDC'], signo='pos', promedio=True, string=True),
                    '<idc_pos>':     get_per_dims(df_sub_2022, dims=['IDC'], signo='pos', promedio=True),
                    '<enl_str_pos>': get_per_dims(df_sub_2022, dims=['ENL'], signo='pos', promedio=True, string=True),
                    '<enl_pos>':     get_per_dims(df_sub_2022, dims=['ENL'], signo='pos', promedio=True),
                    '<sat_str_pos>': get_per_dims(df_sub_2022, dims=['SAT'], signo='pos', promedio=True, string=True),
                    '<sat_pos>':     get_per_dims(df_sub_2022, dims=['SAT'], signo='pos', promedio=True),
                    '<ids_pos>': get_per_dims(df_sub_2022, dims=['IDS'], signo='neg'),    # Invertido
                    '<ids_neu>': get_per_dims(df_sub_2022, dims=['IDS'], signo='neu'),
                    '<ids_neg>': get_per_dims(df_sub_2022, dims=['IDS'], signo='pos'),    # Invertido
                    '<sob_area>': sob_area,
                    '<sob_area_list_pos>': get_per_dims(df_2022[df_2022[ant_seg] == sob_area], dims_glob, signo='pos')
                }

                for s in ['pos', 'neu', 'neg']:
                    data_2022[f'<all_{s}>'] =  get_per_dims(df_sub_2022, dims, signo=s)
                    data_2022[f'<glob_{s}>'] =      get_per_dims(df_2022, dims_glob, signo=s, promedio=True)
                    data_2022[f'<glob_str_{s}>'] =  get_per_dims(df_2022, dims_glob, signo=s, promedio=True, string=True)
                    data_2022[f'<glob_list_{s}>'] = get_per_dims(df_2022, dims_glob, signo=s)

                    data_2022[f'<area_{s}>'] =  get_per_dims(df_sub_2022, dims_glob, signo=s, promedio=True)
                    data_2022[f'<area_str_{s}>'] =  get_per_dims(df_sub_2022, dims_glob, signo=s, promedio=True, string=True)
                    data_2022[f'<area_list_{s}>'] = get_per_dims(df_sub_2022, dims_glob, signo=s)

                    data_2022[f'<sob_area_{s}>'] =  get_per_dims(df_2022[df_2022[ant_seg] == sob_area], dims_glob, signo=s, promedio=True)

                    data_2022[f'<pbd_list_{s}>'] =  get_per_dims(df_sub_2022, dims=['PBD'], signo=s)
                    data_2022[f'<pbd_item_{s}>'] =  get_pts_items(df_sub_2022, dim='pbd', signo=s)
                    data_2022[f'<cfz_list_{s}>'] =  get_per_dims(df_sub_2022, dims=['CFZ'], signo=s)
                    data_2022[f'<cfz_item_{s}>'] =  get_pts_items(df_sub_2022, dim='cfz', signo=s)

                if unico == False:
                    data_2021 = {
                        '<area_pos>': get_per_dims(df_sub_2021, dims_glob, signo='pos', promedio=True),
                        '<area_neu>': get_per_dims(df_sub_2021, dims_glob, signo='neu', promedio=True),
                        '<area_neg>': get_per_dims(df_sub_2021, dims_glob, signo='neg', promedio=True),
                        '<area_list_pos>': get_per_dims(df_sub_2021, dims_glob, signo='pos'),

                        '<enl_pos>':  get_per_dims(df_sub_2021, dims=['ENL'], signo='pos', promedio=True),
                        '<idc_pos>':  get_per_dims(df_sub_2021, dims=['IDC'], signo='pos', promedio=True),
                        '<sat_pos>':  get_per_dims(df_sub_2021, dims=['SAT'], signo='pos', promedio=True)
                    }

                
                # RANKING ITEMS
                top_items_org_pos = top_items(df=df_sub_2022, dict_items=dict_items, dims=dims_org, signo='pos')
                top_items_loc_pos = top_items(df=df_sub_2022, dict_items=dict_items, dims=dims_loc, signo='pos')

                top_items_org_neu = top_items(df=df_sub_2022, dict_items=dict_items, dims=dims_org, signo='neu')
                top_items_loc_neu = top_items(df=df_sub_2022, dict_items=dict_items, dims=dims_loc, signo='neu')

                top_items_org_neg = top_items(df=df_sub_2022, dict_items=dict_items, dims=dims_org, signo='neg')
                top_items_loc_neg = top_items(df=df_sub_2022, dict_items=dict_items, dims=dims_loc, signo='neg')


                # IMPORTANCIA RELATIVA
                dict_dims_per_pos = dict(zip(dims, data_2022['<all_pos>']))

                # CONCLUSIONES
                conclusiones(data=data_2022, dims_glob=dims_glob)


                prs = Presentation('ppts/template.pptx')

                # DIAPO 1
                diapo_replace_txt(prs, data_2022, nro_slide=1, txts=['<area>'])
                # DIAPO 15
                diapo_replace_txt(prs, data_2022, nro_slide=15, txts=['<area>', '<asi_all>', '<asi_yes>', '<asi_not>'])
                diapo_dotacion(prs, data_2022, nro_slide=15, nro_shape=5)
                # DIAPO 16
                diapo_replace_txt(prs, data_2022, nro_slide=16, txts=['<area>', '<area_str10_pos>'])
                diapo_pie(prs, data_2022, nro_slide=16, nro_shape=3, key='area')
                # DIAPO 17
                if j != 'Global Vitacura':
                    diapo_comp_bars(prs, data1=data_2022, data2=data_2022, nro_slide=17, nro_shape=1, key=['sob_area', 'area'], series=['Área', 'Área madre'])
                    txt_replace(prs, data_2022, nro_slide=17, nro_shape=0, txt='<sob_area>')
                    txt_replace(prs, data_2022, nro_slide=17, nro_shape=4, txt='<sob_area>')
                    txt_replace(prs, data_2022, nro_slide=17, nro_shape=5, txt='<area>')
                # DIAPO 18
                if unico == False:
                    diapo_comp_bars(prs, data1=data_2021, data2=data_2022, nro_slide=18, nro_shape=0, key=['area', 'area'], series=['Área', 'Resultados 2021'])
                    txt_replace(prs, data_2022, nro_slide=18, nro_shape=3, txt='<area>')
                    txt_replace(prs, data_2022, nro_slide=18, nro_shape=4, txt='<area>')

                #DIAPO 19
                diapo_dims(prs, data_2022, nro_slide=19, nro_shape=1, dims=dims_glob, key='area_list', signo='pos', comp=data_2022['<area_pos>'])
                diapo_replace_txt(prs, data_2022, nro_slide=19, txts=['<area_str_pos>'])
                # DIAPO 20
                diapo_dims(prs, data_2022, nro_slide=20, nro_shape=4, dims=dims_glob, key='area_list')
                # DIAPO 21
                if j == 'Global Vitacura':
                    data_2022['<comp_20>'] = "Comparación de resultados con el año anterior"
                    diapo_dims(prs, data_2022, nro_slide=21, nro_shape=1, dims=dims_glob, key='area_list', signo='pos', comp=data_2021['<area_list_pos>'], str_comp='Vitacura 2021')
                else:
                    data_2022['<comp_20>'] = "Comparación dimensiones\nporcentajes de apreciación positiva"
                    diapo_dims(prs, data_2022, nro_slide=21, nro_shape=1, dims=dims_glob, key='area_list', signo='pos', comp=data_2022['<sob_area_list_pos>'], str_comp=sob_area, comp2=data_2021['<area_list_pos>'], str_comp2=f"{data_2022['<area>']} 2021")
                diapo_replace_txt(prs, data_2022, nro_slide=21, txts=['<comp_20>'])
                # DIAPO 22
                diapo_dims(prs, data_2022, nro_slide=22, nro_shape=2, dims=dims_glob, key='area_list', signo='pos', comp=list_ant_pub, str_comp='Sector Público', comp2=list_ant_prv, str_comp2='Sector Privado')
                # DIAPO 23
                diapo_dims(prs, data_2022, nro_slide=23, nro_shape=4, dims=['Intención de Salida'], key='ids')
                # DIAPO 24
                diapo_replace_txt(prs, data_2022, nro_slide=24, txts=['<enl_str_pos>', '<idc_str_pos>', '<sat_str_pos>'])
                # DIAPO 25
                if unico == False:
                    diapo_dims_ent_com_sat(prs,data1=data_2021, data2=data_2022, nro_slide=25, nro_shape=5, dims=['Entusiasmo laboral', 'Compromiso organizacional', 'Satisfacción laboral'], signo='pos')

                # DIAPO 26
                diapo_dims(prs, data_2022, nro_slide=26, nro_shape=3, dims=['Providad'], key='pbd_list')
                diapo_items(prs, data_2022, nro_slide=26, nro_shape=2, dict_items=dict_items, item='pbd')
                # DIAPO 27
                diapo_dims(prs, data_2022, nro_slide=27, nro_shape=3, dims=['Confianza'], key='cfz_list')
                diapo_items(prs, data_2022, nro_slide=27, nro_shape=2, dict_items=dict_items, item='cfz')

                if len(sub_areas) > 1:
                    # DIAPO 30
                    diapo_area(prs, df=df_sub_2022, dims=dims_glob, nro_slide=30, nro_shape=1, nxt_seg=nxt_seg, sub_areas=sub_areas, areas_not=areas_not)
                    # DIAPO 31
                    diapo_tabla(prs=prs, df=df_sub_2022, dims=dims_glob, nro_slide=31, nro_shape=0, titulo=j, nxt_seg=nxt_seg, sub_areas=sub_areas, areas_not=areas_not)
                
                if j in areas_glob:
                    # DIAPO 32
                    diapo_car(prs, df=df_sub_2022, nro_car=1, dims=dims_glob, nro_slide=32, nro_shape=1, orden=list_car1)
                    # DIAPO 33
                    diapo_tabla(prs=prs, df=df_sub_2022, nro_slide=33, nro_shape=1, titulo=j, dims=dims_glob, nro_car=1, orden=list_car1)
                    # DIAPO 34
                    diapo_car(prs, df=df_sub_2022, nro_car=2, dims=dims_glob, nro_slide=34, nro_shape=1, orden=list_car2)
                    # DIAPO 35
                    diapo_tabla(prs=prs, df=df_sub_2022, nro_slide=35, nro_shape=1, titulo=j, dims=dims_glob, nro_car=2, orden=list_car2)
                    # DIAPO 36
                    diapo_car(prs, df=df_sub_2022, nro_car=4, dims=dims_glob, nro_slide=36, nro_shape=1)
                    # DIAPO 37
                    diapo_tabla(prs=prs, df=df_sub_2022, nro_slide=37, nro_shape=1, titulo=j, dims=dims_glob, nro_car=4)
                    # DIAPO 38
                    diapo_car(prs, df=df_sub_2022, nro_car=3, dims=dims_glob, nro_slide=38, nro_shape=1, orden=list_car3)
                    # DIAPO 39
                    diapo_tabla(prs=prs, df=df_sub_2022, nro_slide=39, nro_shape=1, titulo=j, dims=dims_glob, nro_car=3, orden=list_car3)
                    # DIAPO 40
                    diapo_car(prs, df=df_sub_2022, nro_car=5, dims=dims_glob, nro_slide=40, nro_shape=1)
                    # DIAPO 41
                    diapo_tabla(prs=prs, df=df_sub_2022, nro_slide=41, nro_shape=1, titulo=j, dims=dims_glob, nro_car=5)
                    # DIAPO 42
                    diapo_car(prs, df=df_sub_2022, nro_car=5, dims=dims_glob, nro_slide=42, nro_shape=1)
                    # DIAPO 43
                    diapo_tabla(prs=prs, df=df_sub_2022, nro_slide=43, nro_shape=1, titulo=j, dims=dims_glob, nro_car=5)
                
                # DIAPO 46
                diapo_rank(prs, nro_slide=46, nro_shape=3, tops=top_items_org_pos, dict_items=dict_items)
                diapo_rank(prs, nro_slide=46, nro_shape=4, tops=top_items_loc_pos, dict_items=dict_items)
                lineas_ap_glob(prs, nro_slide=46, apr_glob=data_2022['<glob_pos>'])

                # DIAPO 47
                diapo_rank(prs, nro_slide=47, nro_shape=3, tops=top_items_org_neu, dict_items=dict_items)
                diapo_rank(prs, nro_slide=47, nro_shape=4, tops=top_items_loc_neu, dict_items=dict_items)
                lineas_ap_glob(prs, nro_slide=47, apr_glob=data_2022['<glob_neu>'])

                # DIAPO 48
                diapo_rank(prs, nro_slide=48, nro_shape=3, tops=top_items_org_neg, dict_items=dict_items)
                diapo_rank(prs, nro_slide=48, nro_shape=4, tops=top_items_loc_neg, dict_items=dict_items)
                lineas_ap_glob(prs, nro_slide=48, apr_glob=data_2022['<glob_neg>'])

                # DIAPO 50
                diapo_ir_txt(prs, nro_slide=50, nro_shape=2, all_dims=dict_dims_per_pos, tops=dict_ir(df_sub_2022, df_imp, dims_org), glob_mean=data_2022['<area_pos>'])
                diapo_ir_txt(prs, nro_slide=50, nro_shape=3, all_dims=dict_dims_per_pos, tops=dict_ir(df_sub_2022, df_imp, dims_loc), glob_mean=data_2022['<area_pos>'])
                txt_replace(prs, data_2022, nro_slide=50, nro_shape=4, txt='<area_str_pos>')

                # DIAPO 64
                diapo_replace_txt(prs, data_2022, nro_slide=64, txts=['<dim_pos1>', '<txt_pos1>', '<per_pos1>', '<dim_pos2>', '<txt_pos2>', '<per_pos2>'])
                # DIAPO 65
                diapo_replace_txt(prs, data_2022, nro_slide=65, txts=['<dim_neg1>', '<txt_neg1>', '<per_neg1>', '<dim_neg2>', '<txt_neg2>', '<per_neg2>'])
                # DIAPO 67
                diapo_dims(prs, data_2022, nro_slide=67, nro_shape=1, dims=dims_glob, key='area_list', signo='pos')
                # DIAPO 69
                diapo_replace_txt(prs, data_2022, nro_slide=69, txts=['<area>'])


                # Eliminación de diapositivas
                del_diapo(prs, area=j, areas_glob=areas_glob, sub_areas=sub_areas, unico=unico)
    

                # Guardar
                prs.save(f"ppts/autorreportes/{j}.pptx")

Global Vitacura
Área Educación
Área Salud
Área Municipalidad
Dirección de Servicios de Salud, Educacion y demas incorporados a la gestion municipal
Dirección de Seguridad Pública
Dirección de Administración y Finanzas
Dirección de Desarrollo Comunitario
Dirección de Comunicaciones, Asuntos Corporativos y Prensa
Dirección de Tránsito y Transporte Público
Dirección de Asesoría Jurídica
Dirección de Obras Municipales
Dirección de Personas
SECPLA
1er Juzgado de Policía Local
Dirección Innovación y Sustentabilidad
Dirección de Infraestructura Comunal
Dirección de Informática
Dirección de Asesoría Urbana
Administración Municipal
Dirección de Medio Ambiente, Aseo y Ornato
2do Juzgado de Policía Local
Secretaría Municipal
Dirección de Control
Colegio Antártica Chilena
Liceo Amanda Labarca
COSAM
Departamento de Administración de Servicios
Subdirección de Administración
Departamento de Asistencia y Acción Social
Departamento de Atención al Vecino
Departamento de Educación
Departamento de Licenci

## __Cálculo de correlaciones__

In [43]:
# matrix = cor_mtx(df_dims_mean, dims=dims_ind_res)
# x = dict_cor_ppt(matrix, dims=dims_ind_res)

# prs = Presentation('ppts/template.pptx')

# diapo_cor(prs, nro_slide=54, dict_cor=x)

# prs.save("ppts/autorreportes/test.pptx")

#### __Revisor de Contenido__

In [44]:
prs = Presentation('ppts/template.pptx')
check_diapo(prs, nro_slide=1, print_text=True)

0 <pptx.shapes.picture.Picture object at 0x00000134E84EED90>
1 <area>
2 Estudio de Clima Organizacional  Municipalidad de Vitacura 2022
3 <pptx.shapes.picture.Picture object at 0x00000134E7B3CD90>
4 <pptx.shapes.picture.Picture object at 0x00000134E84EED90>
5 <pptx.shapes.connector.Connector object at 0x00000134E73047D0>
6 <pptx.shapes.picture.Picture object at 0x00000134E84EED90>
7 <pptx.shapes.picture.Picture object at 0x00000134E73047D0>


## __Cajón de Arena__

#### __Test__

In [45]:
# prs = Presentation('ppts/template.pptx')
# diapo_cor(prs, nro_slide=54, dict_cor=x)
# prs.save("ppts/autorreportes/test.pptx")